In [510]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [520]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors.kde import KernelDensity
import statsmodels.api as sm
import seaborn as sns
import loess
import random
import os
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [646]:
years=range(2000,2019)

In [522]:
# # Ignore this section.
# # Defining Virus tramission temperarture range(in Fahrenheit)
# a=60
# b=95
# # Converting Fahrenheit to Kelvin
# a=(a-32)*5/9 + 273
# b=(b-32)*5/9 + 273

Assigning bin classes to every instance

In [523]:
#Import main WNV data
path_data="/Users/sparshagarwal/Downloads/WNV_challenge/neurownv_by_county.csv"
data=pd.read_csv(path_data)
data.head()

,fips,county,state,location,year,count
0,01001,Autauga,Alabama,Alabama-Autauga,2000,0
1,01001,Autauga,Alabama,Alabama-Autauga,2001,0
2,01001,Autauga,Alabama,Alabama-Autauga,2002,1
3,01001,Autauga,Alabama,Alabama-Autauga,2003,0
4,01001,Autauga,Alabama,Alabama-Autauga,2004,0


In [524]:
#Bin labels for classification
bins={1:"[0,1)",2:"[1,6)", 3:"[6,11)", 4:"[11,16)", 5:"[16,21)", 6:"[21,26)", 7:"[26,31)", 8:"[31,36)", 9:"[36,41)", 10:"[41,46)", 11:"[46,51)", 12:"[51,101)", 13:"[101,151)", 14:"[151,201)", 15:"[201,1000)"}

In [525]:
#Assigning bin labels to instances
bin_assign=[]
bin_values=list(bins.values())
for i in range(len(data)):
    for j in range(len(bin_values)):
        bi=bin_values[j]
        bi_last=int(bi[:-1].split(",")[1])
        if(data.iloc[i]["count"]<bi_last):
            bin_assign.append(j+1)
            break
data["bin"]=bin_assign
data.head()

,fips,county,state,location,year,count,bin
0,01001,Autauga,Alabama,Alabama-Autauga,2000,0,1
1,01001,Autauga,Alabama,Alabama-Autauga,2001,0,1
2,01001,Autauga,Alabama,Alabama-Autauga,2002,1,2
3,01001,Autauga,Alabama,Alabama-Autauga,2003,0,1
4,01001,Autauga,Alabama,Alabama-Autauga,2004,0,1


In [528]:
#Dataframe for bins and labels. Used just for visualizing bins.
bin_df=pd.DataFrame()
bin_df["Class"]=bins.keys()
bin_df["Bin"]=bins.values()
bin_df

,Class,Bin
0,1,"[0,1)"
1,2,"[1,6)"
2,3,"[6,11)"
3,4,"[11,16)"
4,5,"[16,21)"
5,6,"[21,26)"
6,7,"[26,31)"
7,8,"[31,36)"
8,9,"[36,41)"
9,10,"[41,46)"


In [529]:
data.to_csv("/Users/sparshagarwal/Desktop/NCSA/Dataframes/WNV_challenge.csv", index=False)

Converting the bins into one-hot vector notation

In [530]:
#Loading the data with State Avg column

path_data="/Users/sparshagarwal/Desktop/NCSA/Dataframes/WNV_challenge_neighCountyAvg.csv"
data=pd.read_csv(path_data)

#Changes/Cleaning that needs to be done in data as mentioned by the cdc manual

data['county'] = np.where(data['county']=='Bedford/Bedford City', 'Bedford', data['county'])
data['fips'] = np.where(data['fips']=='51019/51515', '51019', data['fips'])
data['location'] = np.where(data['location']=='Virginia-Bedford/Bedford City', 'Virginia-Bedford', data['location'])

data['county'] = np.where(data['county']=='Oglala Lakota/Shannon', 'Oglala Lakota', data['county'])
data['fips'] = np.where(data['fips']=='46102/46113', '46102', data['fips'])
data['location'] = np.where(data['location']=='South Dakota-Oglala Lakota/Shannon', 'South Dakota-Oglala Lakota', data['location'])

#converting fips from object/string to int datatype

data['fips']=data['fips'].apply(pd.to_numeric)
data.rename(columns={'fips':'GEOID'}, inplace=True)
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0


In [531]:
#For converting bin labels to dummy variables
bin_dummy={1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[], 10:[], 11:[], 12:[], 13:[], 14:[], 15:[]}
for i in range(len(data)):
    cl=data.iloc[i]["bin"]
    for j in bin_dummy:
        bin_dummy[j].append(0)
    bin_dummy[cl][-1]=1

In [532]:
for i in bin_dummy:
    data[i]=bin_dummy[i]

In [533]:
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,6,7,8,9,10,11,12,13,14,15
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0,0,0,0,0
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0,0,0,0,0
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0


Adding gini index data

In [536]:
path_data_var="/Users/sparshagarwal/Downloads/NARR_weather_data/weekly_precipitation_gini.csv"
data_var=pd.read_csv(path_data_var)
data_var.head()

,GEOID,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,31039,0.590263,0.612149,0.625033,0.634139,0.574620,0.610425,0.611561,0.575513,0.572891,0.569512,0.609886,0.598652,0.613702,0.597952,0.648474,0.552040,0.627931,0.683478,0.560006
1,53069,0.509748,0.611138,0.651749,0.581479,0.526376,0.643554,0.657909,0.633189,0.581886,0.627300,0.488065,0.567925,0.586592,0.548523,0.599406,0.667290,0.562455,0.581535,0.605823
2,35011,0.738054,0.653759,0.710302,0.682077,0.669582,0.625178,0.725630,0.557457,0.710686,0.659868,0.619986,0.720013,0.698934,0.753707,0.695405,0.570813,0.633473,0.693954,0.642412
3,31109,0.592946,0.565162,0.685569,0.614409,0.562373,0.612657,0.620892,0.585818,0.606183,0.543172,0.576916,0.571919,0.659648,0.623640,0.625107,0.561466,0.611194,0.683340,0.598829
4,31129,0.576154,0.584979,0.686581,0.678051,0.554531,0.584689,0.607414,0.566429,0.609132,0.532402,0.585935,0.603130,0.649098,0.572670,0.626007,0.639256,0.614907,0.676862,0.622858


In [537]:
#Adding gini index data to the main data frame
gini=[]
temp=data[data["year"]==2000]
for i in range(len(temp)):
    geoid=temp.iloc[i]["GEOID"]
    gini.extend(data_var[data_var["GEOID"]==geoid].values[0][1:])
data["Gini"]=gini
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,7,8,9,10,11,12,13,14,15,Gini
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0.523796
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0.547524
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0,0,0,0,0.420538
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0,0,0,0,0.503949
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0.519989


In [216]:
#This are the missing values in socioeconomic data
data_var[data_var.isnull()["2000"]==True]

,GEOID,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
421,51133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
679,25019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
844,24009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
933,55089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2149,53055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3070,51103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Adding air, precipitation and humidity data.

In [255]:
#Defining number of days in each year
days_dict={}
for year in years:
    days=365
    if(year%4==0):
        days=366
    days_dict[year]=days

In [538]:
pd.read_csv("/Users/sparshagarwal/Downloads/NARR_weather_data/air.sfc_complete.csv").head()

,GEOID,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
0,31039,273.575348,273.152313,270.045654,263.826019,263.761169,267.697021,266.799683,268.170532,269.215179,...,271.670746,272.377045,271.251678,272.059662,272.300720,274.062469,267.594421,261.499512,266.040192,268.401001
1,53069,278.151520,277.492157,278.041748,280.322113,278.388123,275.175537,278.803589,279.602173,278.135101,...,278.930511,280.261810,280.073944,277.438568,277.962830,277.714813,278.848328,282.222168,279.360504,275.225220
2,35011,280.982574,278.097626,275.733154,272.142426,274.557068,274.793701,274.357300,274.543579,275.445648,...,281.151215,278.042084,277.648163,279.322357,282.034119,280.431610,274.013367,270.857910,268.842926,270.201782
3,31109,274.863749,274.753886,269.716227,265.936707,264.696737,269.295329,268.118693,271.154256,272.619486,...,273.343282,273.247498,273.062236,274.025818,274.513631,278.983053,270.374390,264.951335,267.426900,270.778605
4,31129,275.518707,275.363251,271.459717,268.804535,268.267029,271.560303,271.243042,272.885376,272.832367,...,274.237152,274.195404,274.302460,274.026459,275.515564,277.624969,270.354187,265.688965,267.926910,269.496704


In [539]:
pd.read_csv("/Users/sparshagarwal/Downloads/NARR_weather_data/apcp_complete.csv").head()

,GEOID,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
0,31039,0.030561,0.533203,0.190244,0.024681,0.380859,0.127604,0.002239,0.003889,0.025658,...,0.000000,0.002917,0.023438,0.010900,7.646484,32.041016,0.511719,0.015625,0.020298,0.146484
1,53069,13.407515,11.417969,17.592587,28.305931,2.996094,2.277344,3.859661,12.140607,22.058861,...,3.175781,29.905258,4.234375,0.006994,8.492188,3.652344,8.917969,24.777344,18.367954,0.125000
2,35011,0.026655,0.012695,0.027158,0.005150,0.000000,0.007812,0.000286,0.003889,0.004174,...,0.000000,0.012683,0.007812,0.090978,2.303711,5.824219,1.153320,2.227539,0.003696,0.250000
3,31109,0.016889,0.244141,0.139463,0.024681,0.031250,0.000000,0.000286,0.003889,0.025007,...,0.000000,0.009428,0.065104,0.163244,10.221354,25.723958,0.432292,0.000000,0.055454,0.118490
4,31129,0.053998,0.125000,0.110166,0.087181,0.027344,0.000000,0.000286,0.003889,0.035424,...,0.048828,0.137683,0.070312,0.006994,19.931641,18.550781,0.018229,0.000000,0.010532,0.058594


In [540]:
pd.read_csv("/Users/sparshagarwal/Downloads/NARR_weather_data/rhum.2m_complete.csv").head()

,GEOID,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,...,2018-12-22,2018-12-23,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,2018-12-30,2018-12-31
0,31039,71.160995,83.196068,90.932648,77.843132,89.490936,89.135612,81.607300,83.936584,84.790359,...,85.344955,68.315140,77.672234,88.540977,94.371048,95.334702,78.557739,83.107796,88.273376,90.270866
1,53069,92.455917,92.123802,94.061554,93.477898,92.729218,91.659050,94.511597,92.768616,93.893875,...,88.102768,92.817093,93.502312,91.390587,90.605423,92.487045,94.862427,95.984749,92.068298,89.966179
2,35011,44.870960,37.532986,38.796906,32.387077,30.112026,44.660027,51.894409,47.045956,41.455402,...,29.235573,48.041702,58.362671,39.840786,71.350540,58.571030,63.235470,79.892952,75.186462,64.512077
3,31109,76.083527,80.280055,90.226919,79.330765,90.020884,86.023631,80.663940,80.975647,87.830068,...,83.387929,69.671254,79.618200,86.101522,93.104116,94.557363,70.120239,79.755580,84.229757,87.898468
4,31129,68.399284,68.996849,81.002960,59.604851,76.719452,68.780144,60.124878,68.989319,77.204422,...,75.844955,72.977249,78.881218,80.785118,92.511673,90.071030,68.872192,77.379280,78.996033,85.610710


The code just below this takes a lot of time, and therefore is commented out so that it does not run. The results obtained after running the code below are stored in a csv file/dataframe which is imported everytime when it is to be used.

In [314]:
# #Taking the average values of the variables across a single year
# df_temp=pd.DataFrame()
# df_prec=pd.DataFrame()
# df_hum=pd.DataFrame()

# variables=["Temp","Prec","Hum"]

# for var in variables:
#     value=[]
#     if(var=="Temp"):
#         path_data_var="/Users/sparshagarwal/Downloads/NARR_weather_data/air.sfc_complete.csv"
#     if(var=="Prec"):
#         path_data_var="/Users/sparshagarwal/Downloads/NARR_weather_data/apcp_complete.csv"
#     if(var=="Hum"):
#         path_data_var="/Users/sparshagarwal/Downloads/NARR_weather_data/rhum.2m_complete.csv"
#     data_var=pd.read_csv(path_data_var)
    
#     year_col=[]
#     geoid_col=[]
#     for i in range(len(data_var)):
#         print(i)
#         count=1
#         for year in years:
#             geoid_col.append(data_var.iloc[i][0])
#             year_col.append(year)
#             days=days_dict[year]
    
#             avg=np.mean(list(data_var.iloc[i][count:count+days]))
#             value.append(avg)
            
#             count+=days
    
#     if(var=="Temp"):
#         df_temp["GEOID"]=geoid_col
#         df_temp["year"]=year_col
#         df_temp["Temp"]=value
        
#     if(var=="Prec"):
#         df_prec["GEOID"]=geoid_col
#         df_prec["year"]=year_col
#         df_prec["Prec"]=value
        
#     if(var=="Hum"):
#         df_hum["GEOID"]=geoid_col
#         df_hum["year"]=year_col
#         df_hum["Hum"]=value
        


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [542]:
df_temp.head()

,GEOID,year,Temp
0,31039.0,2000,284.998561
1,31039.0,2001,283.795406
2,31039.0,2002,284.335251
3,31039.0,2003,284.039300
4,31039.0,2004,284.294354


In [543]:
#Merging all the dataframes together
variables=["Temp","Prec","Hum"]
for var in variables:
    if(var=="Temp"):
        df_c=df_temp.copy()
    if(var=="Prec"):
        df_c=df_prec.copy()
    if(var=="Hum"):
        df_c=df_hum.copy()
        
    data=pd.merge(data, df_c,  how='inner', left_on=['GEOID','year'], right_on = ['GEOID','year'])

In [546]:
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,10,11,12,13,14,15,Gini,Temp,Prec,Hum
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0,0.523796,292.039507,2.707358,68.738111
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0,0.547524,291.090697,3.960096,75.034695
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0,0.420538,291.832848,3.678712,76.543474
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0,0.503949,291.327859,4.100839,77.908219
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0,0.519989,291.911606,3.166458,75.178524


Adding County_type column (urban/rural)

In [547]:
# Path to the file with WNV data used to classify urban and rural counties
wnv_file_path= "/Users/sparshagarwal/Downloads/WMV_data/Arbovirus_risk_modeling_US/WNV_human_cases/WNV_NI_NNI_1999to2015_prevalence_incidence_final_20180530.csv"
#Adding County_type column
data_old=pd.read_csv(wnv_file_path, encoding='latin-1')
data_old.rename(columns={'GEOID10':'GEOID', 'Select_County':'County_type'}, inplace=True)
data_old=data_old[["GEOID","County_type"]]
data=pd.merge(data, data_old,  how='inner', left_on=['GEOID'], right_on = ['GEOID'])

In [549]:
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,11,12,13,14,15,Gini,Temp,Prec,Hum,County_type
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0.523796,292.039507,2.707358,68.738111,1
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0.547524,291.090697,3.960096,75.034695,1
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0.420538,291.832848,3.678712,76.543474,1
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0.503949,291.327859,4.100839,77.908219,1
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0.519989,291.911606,3.166458,75.178524,1


In [552]:
# #To store the data so that so taht socioeconomic data does not have be to added again.
# data.to_csv("/Users/sparshagarwal/Desktop/NCSA/Dataframes/Data_merged.csv", index=False)

In [647]:
data=pd.read_csv("/Users/sparshagarwal/Desktop/NCSA/Dataframes/Data_merged.csv")
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,11,12,13,14,15,Gini,Temp,Prec,Hum,County_type
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0.523796,292.039507,2.707358,68.738111,1
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0.547524,291.090697,3.960096,75.034695,1
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0.420538,291.832848,3.678712,76.543474,1
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0.503949,291.327859,4.100839,77.908219,1
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0.519989,291.911606,3.166458,75.178524,1


Adding introduction/outbreak year nominal variable

In [648]:
# Adding a column for incident year (first year of WNV introduction) and incident year nominal variable in the previous dataframe
df_unique=data[data["year"]==2000]
incident_year_combine=[]
incident_year_nominal_combine=[]
for j in range(len(df_unique)):
    incident_year=[years[-1] for year in years]
    incident_year_nominal=[1 for year in years]
    geo_id=df_unique.iloc[j]["GEOID"]
    df_temporary=data[data["GEOID"]==geo_id]
    for i in range(len(years)):  
        if(df_temporary.iloc[i]["count"]!=0):
            incident_year=[df_temporary.iloc[i]["year"] for year in years]
            incident_year_nominal[i]=0
            break
        incident_year_nominal[i]=0
    incident_year_combine.extend(incident_year)
    incident_year_nominal_combine.extend(incident_year_nominal)
data["Incident_year"]=incident_year_combine
data["Incident_nominal"]=incident_year_nominal_combine

In [558]:
pd.read_csv("/Users/sparshagarwal/Downloads/WMV_data/Socioeconomics/se_data.csv").head()

,YEAR,Areaname,STCOU,Median_Household_Income,Poverty_number_of_people,Poverty_percent_of_people,Public_school_enrollment_Fall
0,1995,UNITED STATES,0,34076,36424609,13.8,43369917
1,1995,ALABAMA,1000,27357,752097,17.6,730509
2,1995,"Autauga, AL",1001,34842,5471,13.4,7381
3,1995,"Baldwin, AL",1003,35014,15494,12.4,19384
4,1995,"Barbour, AL",1005,24423,6423,24.9,5111


In [559]:
pd.read_csv("/Users/sparshagarwal/Downloads/WMV_data/Socioeconomics/race_data.csv").head()

,YEAR,Areaname,STCOU,Resident_population_White_alone,Resident_population_White_alone_percent,Resident_population_Black_alone,Resident_population_Black_alone_percent,Resident_population_American_Indian_and_Alaska_Native_alone,Resident_population_American_Indian_and_Alaska_Native_alone_percent,Resident_population_Asian_alone,Resident_population_Asian_alone_percent,Resident_population_Native_Hawaiian_and_Other_Pacific_Islander_alone,Resident_population_Native_Hawaiian_and_Other_Pacific_Islander_alone_percent,Resident_population_Two_or_more_races,Resident_population_Two_or_more_races_percent,Resident_population_Hispanic_or_Latino_Origin,Resident_population_Hispanic_or_Latino_Origin_percent,Resident_population_Not_Hispanic_White_alone,Resident_population_Not_Hispanic_White_alone_percent
0,2000,UNITED STATES,0,228602436,81.0,35807824,12.7,2675900,0.9,10687312,3.8,465614,0.2,3932871,1.4,35642379,12.6,195762792,69.4
1,2000,ALABAMA,1000,3197029,71.8,1161485,26.1,23038,0.5,32446,0.7,1531,0.0,36320,0.8,76504,1.7,3131423,70.3
2,2000,"Autauga, AL",1001,35549,81.0,7558,17.2,198,0.5,204,0.5,13,0.0,350,0.8,610,1.4,35018,79.8
3,2000,"Baldwin, AL",1003,124101,87.8,14678,10.4,823,0.6,547,0.4,38,0.0,1171,0.8,2486,1.8,121868,86.2
4,2000,"Barbour, AL",1005,15151,52.2,13482,46.4,131,0.5,87,0.3,9,0.0,175,0.6,478,1.6,14783,50.9


Adding socioeconomic data for 10 years. The code below is run only if socio economic data is added. Adding socioeconomic data reduces the year range from 2000-2018 to 2000-2009.

In [556]:
variables=["Resident_population_White_alone_percent", "Median_Household_Income", "Poverty_percent_of_people"]
years=range(2000,2010)
for variable in variables:
    if(variable=="Poverty_percent_of_people" or variable=="Median_Household_Income"):
        data_path="/Users/sparshagarwal/Downloads/WMV_data/Socioeconomics/se_data.csv"
    if(variable=="Resident_population_White_alone_percent"):
        data_path="/Users/sparshagarwal/Downloads/WMV_data/Socioeconomics/race_data.csv"
    # Adding data for the variable
    s_data=pd.read_csv(data_path)
    s_data.rename(columns={'STCOU':'GEOID', 'YEAR':'year'}, inplace=True)
    s_data=s_data[s_data["GEOID"]!=0]
    s_data=s_data[s_data["year"]>=2000]
    s_data=s_data[["year", "GEOID", variable]]
    data=pd.merge(data, s_data,  how='inner', left_on=['GEOID','year'], right_on = ['GEOID','year'])

In [560]:
data.head()

,GEOID,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,14,15,Gini,Temp,Prec,Hum,County_type,Resident_population_White_alone_percent,Median_Household_Income,Poverty_percent_of_people
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0.523796,292.039507,2.707358,68.738111,1,81.0,42463,10.5
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0.547524,291.090697,3.960096,75.034695,1,80.9,42183,10.8
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0.420538,291.832848,3.678712,76.543474,1,80.8,42841,10.3
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0.503949,291.327859,4.100839,77.908219,1,80.6,44194,10.4
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0.519989,291.911606,3.166458,75.178524,1,80.4,45379,11.6


In [ ]:
##This code is for adding weather data in CDD range but takes infinite time so ignore this part.

# path_data_temp="/Users/sparshagarwal/Downloads/NARR_weather_data/air.sfc_complete.csv"
# path_data_prec="/Users/sparshagarwal/Downloads/NARR_weather_data/apcp_complete.csv"
# path_data_hum="/Users/sparshagarwal/Downloads/NARR_weather_data/rhum.2m_complete.csv"
# data_temp=pd.read_csv(path_data_temp)
# data_prec=pd.read_csv(path_data_prec)
# data_hum=pd.read_csv(path_data_hum)

# df_temp=pd.DataFrame()
# df_prec=pd.DataFrame()
# df_hum=pd.DataFrame()

# temp=[]
# prec=[]
# hum=[]

# year_col=[]
# geoid_col=[]
# for i in range(len(data_temp)):
#     print(i)
#     count=0
#     for year in years:
#         geoid_col.append(data_temp.iloc[i][0])
#         year_col.append(year)
#         days=days_dict[year]
#         values_temp=[]
#         values_prec=[]
#         values_hum=[]
#         for day in range(days):
#             count+=1
#             if(data_temp.iloc[i][count]>=a and data_temp.iloc[i][count]<=b):
#                 values_temp.append(data_temp.iloc[i][count])
#                 values_prec.append(data_prec.iloc[i][count])
#                 values_hum.append(data_hum.iloc[i][count])
#         temp.append(np.mean(values_temp))
#         prec.append(np.mean(values_prec))
#         hum.append(np.mean(values_hum))

# df_temp["GEOID"]=geoid_col
# df_prec["GEOID"]=geoid_col
# df_hum["GEOID"]=geoid_col

# df_temp["Year"]=year_col
# df_prec["Year"]=year_col
# df_hum["Year"]=year_col

# df_temp["Temp"]=temp
# df_prec["Prec"]=prec
# df_hum["Hum"]=hum

    

Constructing temporal dataframe for different number of prior years.

In [649]:
#Number of previous years used for prediction
num_years=10

In [650]:
    #Features/columns that will be used for prediction.
    #features=["count","neighborCountyAvg", "Gini", "Temp", "Prec", "Hum","County_type","Incident_nominal","Resident_population_White_alone_percent", "Median_Household_Income", "Poverty_percent_of_people"]
    features=["count","neighborCountyAvg", "Gini", "Temp", "Prec", "Hum", "County_type","Incident_nominal"]

In [651]:
#Minimum and maximum value of years that will form the target class
min_year=years[0]+num_years+1
max_year=years[-1]

In [652]:
add_features=features.copy()
add_features.extend(["year","location"])
add_features.extend(str(i) for i in range(1,16))

In [653]:
temp_df=data[add_features]

In [654]:
#Removing the rows having NaN Gini index values
temp_df=temp_df[temp_df.isnull()["Gini"]==False]
temp_df.head()

,count,neighborCountyAvg,Gini,Temp,Prec,Hum,County_type,Incident_nominal,year,location,...,6,7,8,9,10,11,12,13,14,15
0,0,0.0,0.523796,292.039507,2.707358,68.738111,1,0,2000,Alabama-Autauga,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,0.547524,291.090697,3.960096,75.034695,1,0,2001,Alabama-Autauga,...,0,0,0,0,0,0,0,0,0,0
2,1,1.6,0.420538,291.832848,3.678712,76.543474,1,0,2002,Alabama-Autauga,...,0,0,0,0,0,0,0,0,0,0
3,0,0.4,0.503949,291.327859,4.100839,77.908219,1,1,2003,Alabama-Autauga,...,0,0,0,0,0,0,0,0,0,0
4,0,0.0,0.519989,291.911606,3.166458,75.178524,1,1,2004,Alabama-Autauga,...,0,0,0,0,0,0,0,0,0,0


In [655]:
#Creating dataframe for training data
temporal_df=pd.DataFrame() #The final dataframe
for year in range(max_year,min_year-1,-1):
    col="A"
    yearly_df=pd.DataFrame()
    target=temp_df[temp_df["year"]==year]
    classes=[str(i) for i in range(1,16)]
    target=target[classes]
    for prior_year in range(year-2,year-num_years-2,-1):
        df_prior_year=temp_df[temp_df["year"]==prior_year]
        for feature in features:
            feat_values=list(df_prior_year[feature])
            yearly_df[col]=feat_values
            col=chr(ord(col)+1)
    yearly_df=pd.concat([yearly_df.reset_index(),target.reset_index()], axis=1).drop(["index"], axis=1)
    temporal_df=temporal_df.append(yearly_df)

In [656]:
#Creating dataframe for testing data. Ignore this part for now.
year=2020
col="A"
temporal_df_test=pd.DataFrame()
for prior_year in range(year-2,year-num_years-2,-1):
    df_prior_year=temp_df[temp_df["year"]==prior_year]
    for feature in features:
        feat_values=list(df_prior_year[feature])
        temporal_df_test[col]=feat_values
        col=chr(ord(col)+1)

In [657]:
#Exporting data so that it can be imported in another jupyter notebook.
temporal_df.to_csv("/Users/sparshagarwal/Desktop/NCSA/Dataframes/Temporal_df.csv", index=False)
temporal_df_test.to_csv("/Users/sparshagarwal/Desktop/NCSA/Dataframes/Temporal_df_test.csv", index=False)

In [658]:
temporal_df.head()

,A,B,C,D,E,F,G,H,I,J,...,6,7,8,9,10,11,12,13,14,15
0,1,0.000000,0.544365,292.763352,3.006250,72.683919,1,1,0,0.2,...,0,0,0,0,0,0,0,0,0,0
1,2,0.666667,0.529595,293.719919,4.002598,78.128852,1,1,0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.000000,0.510778,293.550745,2.879374,72.633073,0,1,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.500000,0.541940,292.541822,2.947005,73.039254,0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0.333333,0.555216,291.308402,2.702201,70.986648,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
